# Paper Title: "Premonition: Using Generative Models to Preempt Future Data Changes in Continual Learning"

## Demonstration of Premonition initial steps: Realm --> SD Prompts

### About This Python Jupyter Notebook:
- This notebook demonstrates the steps in Premonition between the initial step of providing a realm name, up until the createion of prompts to be passed to Stable Diffusion, for synthetic image generation for the Realm.
- We need to use a Realm name to prompt GPT-4 to generate lists of classes, descriptions and Stable Diffusion prompts .
- We use 6 example Realms, each of which is defined by a short phrase or single word:
    1. "Items near roads"
    2. "Domestic furniture"
    3. "Plants"
    4. "Insects"
    5. "Birds"
    6. "Food"
- We have two separate kinds of Realm: (i) those based on biology, like "Birds", and (ii) more general concepts.
- This is because for fine grained biological classification, we want diversity of species. For more more general concept, we ask the LLM to more generally list types/kinds/example of subtypes within the realm. 
- In order to ensure LLM outputs are easily parsed, we use few-shot prompt engineering, where we provide some exemplars.
- We also prompt the LLM to provide formatted outputs, namely csv formatted.

In [1]:
import pandas as pd
from io import StringIO
from IPython.display import display
from openai import OpenAI


## OpenAI setup:

In [2]:
api_key='copy-your-key-to-here'
client = OpenAI(api_key=api_key)

## LLM model parameters:

In [3]:
MODEL = "gpt-4-turbo-preview" #as at March 2024, uses gpt-4-0125-preview, cost $10.00 / 1M tokens input and $30.00 / 1M tokens output
model_temperature=0.2 #"more focused and deterministic"
model_max_tokens=2000

## Premonition parameters:

In [4]:
object_type_count=20 #increase this for more diversity
classes_per_type_count_biological=20 #for biological species, number of classes depends on number of scientific orders returned
classes_per_type_count_object=10 #for object_type_count=20, should result in 200 classes
max_description_chars=180 #based on current Stable Diffusion models operating better with short descriptions
num_subtypes_each_realm=2 #needed only for testing, to speed things up

## Initial system prompts: conditioned on a *realm*, we want to generate lists of subtypes, from which we will generate classes
    - we use "scientific orders" for biological realms
    - we use "subcategories" for other realm concepts

In [5]:
initial_biological_system_prompt="List the latin names of all scientific orders of {}, ensuring every item in the list is unique. "
initial_biological_system_prompt+="Do not use extra explanations like `Please note that this is not an exhaustive list.' "
initial_biological_system_prompt+="If the provided scientific order was already asked of you, simply repeat your previous answer for it. " # in case of exemplar
initial_object_system_prompt="For each provided prompt, provide a list of {} of the most common subcategories, ensuring every item in the list is unique.".format(object_type_count)
initial_object_system_prompt+="Do not use extra explanations like `Please note that this is not an exhaustive list.' "
initial_object_system_prompt+="If the provided prompt was already asked of you, simply repeat your previous answer for it. " # in case of exemplar


## System prompts to generate classes and descriptions for the subtypes/scientific orders:

In [6]:
biological_system_prompt="I want you to act as an input text prompt for the generative image model called Stable Diffusion. "
biological_system_prompt+="I will give you the scientific name of an order of {0}. "
biological_system_prompt+="Your job is to provide a numbered list of {1} unique randomly chosen *non-extinct* species from the provided order, "
biological_system_prompt+="and for each species give a detailed descriptions in {2} characters or less of the visual characteristics "
biological_system_prompt+="that will help to tell it from other species in a photograph. "
biological_system_prompt+="Also provide each species' latin scientific name, and if known, the common name. "
biological_system_prompt+="Provide them in csv format using ; as the separator with the following columns: item_id, species_latin_name, species_common_name, species_description. "
biological_system_prompt+="Do not use ; within any text fields. "
biological_system_prompt+="If the order has less than {1} species, list all the species using a shorter list. "
biological_system_prompt+="Do not list extinct species or any species from orders different to the one I provide. "
biological_system_prompt+="Do not explain if the order has less than {1} species. "
biological_system_prompt+="Ensure every listed species is only listed once. "
biological_system_prompt+="Do not explain using a note of the form `Note: The X order only contains these Y species.' "
biological_system_prompt+="Do not use ** in the result. "
biological_system_prompt+="If the same species was already asked of you, simply repeat your previous answer for it. " #in case of exemplar
biological_system_prompt+="Describe the visual characteristics and leave out how they sound or smell. "

object_system_prompt="I want you to act as an input text prompt for the generative image model called Stable Diffusion. "
object_system_prompt+="I will give you the name of a physical object/concept related to the theme of {0}. "
object_system_prompt+="Your job is to provide a numbered list of {1} unique randomly chosen types/kinds/examples of the object/concept, "
object_system_prompt+="and for each species give a detailed descriptions in {2} characters or less of the visual characteristics "
object_system_prompt+="that will help to tell it from other related items in a photograph. "
object_system_prompt+="Provide them in csv format using ; as the separator with the following columns: item_id, item_name, item_description. "
object_system_prompt+="Do not use ; within any text fields. "
object_system_prompt+="If the object/concept has less than {1} unique types/kinds/examples, list all of them using a shorter list. "
object_system_prompt+="Do not list items unrelated to the object/concept to the one I provide. "
object_system_prompt+="Do not explain if the object/concept has less than {1} type/kind/example. "
object_system_prompt+="Ensure every listed item is only listed once. "
object_system_prompt+="Do not use ** in the result. "
object_system_prompt+="If the same item was asked of you, simply repeat your previous answer for it. " #in case of exemplar
object_system_prompt+="Describe the visual characteristics and leave out how they sound, smell, taste or feel. "

## To help ensure the model formats results as desired, we will use Few Shot Prompting
- Few Shot prompting will be based on the following exemplars:

In [7]:
exemplar_realms=["Birds","Food"]
exemplar_is_biological=[True,False]
exemplar_realms_singular=["Bird","Food"]
exemplar_bird_orders=", ".join(["casuariiformes", "tinamiformes", "struthioniformes", "rheiformes", "apterygiformes", "anseriformes", "galliformes", "caprimulgiformes", "steatornithiformes", "nyctibiiformes", "podargiformes", "aegotheliformes", "apodiformes", "musophagiformes", "otidiformes", "cuculiformes", "mesitornithiformes", "pterocliformes", "columbiformes", "gruiformes", "podicipediformes", "phoenicopteriformes", "charadriiformes", "eurypygiformes", "phaethontiformes", "gaviiformes", "sphenisciformes", "procellariiformes", "ciconiiformes", "suliformes", "pelecaniformes", "opisthocomiformes", "accipitriformes", "strigiformes", "coliiformes", "leptosomiformes", "trogoniformes", "bucerotiformes", "coraciiformes", "piciformes", "cariamiformes", "falconiformes", "psittaciformes", "passeriformes"])
exemplar_bird_species="item_id;species_latin_name;species_common_name;species_description\n1;Tinamus major;Great Tinamou;Dark greyish-blue bird with a long neck and small head. It has a short tail and strong legs.\n2;Tinamus osgoodi;Black Tinamou;A large, dark bird with a long neck and small head. It has a short tail and strong legs.\n3;Tinamus solitarius;Solitary Tinamou;A medium-sized bird with a brownish-grey body, a long neck and small head.\n4;Tinamus tao;Grey Tinamou;A large bird with a grey body, long neck and small head. It has a short tail and strong legs.\n5;Tinamus guttatus;White-throated Tinamou;A small bird with a white throat, brown body, long neck and small head.\n6;Crypturellus obsoletus;Brown Tinamou;A medium-sized bird with a brown body, long neck and small head. It has a short tail and strong legs.\n7;Crypturellus cinnamomeus;Thicket Tinamou;A small bird with a cinnamon-colored body, long neck and small head.\n8;Crypturellus soui;Little Tinamou;A small bird with a greyish-brown body, long neck and small head.\n9;Crypturellus undulatus;Undulated Tinamou;A medium-sized bird with a brown body, long neck and small head. It has a short tail and strong legs.\n10;Crypturellus atrocapillus;Black-capped Tinamou;A small bird with a black cap, brown body, long neck and small head.\n11;Crypturellus transfasciatus;Pale-browed Tinamou;A medium-sized bird with a pale brow, brown body, long neck and small head.\n12;Crypturellus kerriae;Choco Tinamou;A medium-sized bird with a brown body, long neck and small head. It has a short tail and strong legs.\n13;Crypturellus strigulosus;Brazilian Tinamou;A medium-sized bird with a brown body, long neck and small head. It has a short tail and strong legs.\n14;Crypturellus duidae;Grey-legged Tinamou;A medium-sized bird with grey legs, brown body, long neck and small head.\n15;Crypturellus noctivagus;Yellow-legged Tinamou;A medium-sized bird with yellow legs, brown body, long neck and small head.\n16;Crypturellus variegatus;Variegated Tinamou;A medium-sized bird with a variegated brown body, long neck and small head.\n17;Crypturellus brevirostris;Rusty Tinamou;A medium-sized bird with a rusty brown body, long neck and small head.\n18;Crypturellus parvirostris;Small-billed Tinamou;A small bird with a small bill, brown body, long neck and small head.\n19;Crypturellus tataupa;Tataupa Tinamou;A small bird with a brown body, long neck and small head. It has a short tail and strong legs.\n20;Crypturellus casiquiare;Barred Tinamou;A medium-sized bird with a barred brown body, long neck and small head."
exemplar_food_types="Appetizer, Main course, side dish, salad, soup, dessert, snack, breakfast, brunch, lunch, dinner, supper, hors d'oeuvres, amuse-bouche, tapas, antipasto, canapes, finger food, afternoon tea, midnight snack"
exemplar_item="item_id;item_name;item_description\n1;Chesterfield Sofa;Characterized by deep button tufting, rolled arms, and equal back and arm height.\n2;Cabriole Sofa;Features an exposed wooden frame, continuous curved back that forms into the armrests.\n3;Camelback Sofa;Identified by its arched back that peaks in the center and drops in a continuous line into the arms.\n4;Lawson Sofa;Designed for comfort, with a high back, low arms, and soft, overstuffed cushions.\n5;Tuxedo Sofa;Defined by its boxy shape, high arms level with the back, and upholstered with fabric or leather.\n6;English Rolled Arm Sofa;Known for low arms in relation to a high back, with deep seats and high comfort level.\n7;Bridgewater Sofa;Casual and comfortable with softly rolled back, low set back arms, and loose cushions.\n8;Mid-Century Modern Sofa;Features clean lines, geometric shapes, and furniture tilted on thin legs for a minimalist look.\n9;Sectional Sofa;Modular, often L-shaped or U-shaped, and comes in various sections for flexible arrangement.\n10;Sleeper Sofa;A functional sofa that can be converted into a bed, often featuring a fold-out mechanism."

## Functions for generating classes and descriptions from LLM:

In [8]:
def get_subcategories_for_a_realm(realm_description,is_biological=False):
    if is_biological:
        #for fine grained living objects, we use scientific orders to get good diversity
        response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": initial_biological_system_prompt.format(realm_description)},
                    {"role": "user", "content": "Birds"},
                    {"role": "assistant", "content": exemplar_bird_orders},
                    {"role": "user", "content": realm_description},
                ],
                temperature=model_temperature,
                max_tokens=model_max_tokens,
            )
        return response.choices[0].message.content
    else:
        response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": initial_object_system_prompt},
                    {"role": "user", "content": "Food"},
                    {"role": "assistant", "content": exemplar_food_types},
                    {"role": "user", "content": realm_description},
                ],
                temperature=model_temperature, # more focused and deterministic
                max_tokens=model_max_tokens
            )
        return response.choices[0].message.content
    
def get_classes_and_descriptions_from_subcategory(realm,subtype,system_prompt,is_biological):
    if is_biological:
        response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": "tinamiformes"},
                    {"role": "assistant", "content": exemplar_bird_species},
                    {"role": "user", "content": subtype},
                ],
                temperature=model_temperature,
                max_tokens=model_max_tokens
            )
        df = pd.read_csv(StringIO(response.choices[0].message.content),sep=';')
        return df
    else:
        response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": "Sofas"},
                    {"role": "assistant", "content": exemplar_item},
                    {"role": "user", "content": subtype},
                ],
                temperature=model_temperature,
                max_tokens=model_max_tokens
            )
        df = pd.read_csv(StringIO(response.choices[0].message.content),sep=';')
        return df

def get_classes_from_realm(realm,is_bio_realm):
    if realm not in exemplar_realms: #we already have this constants for the subcategories for the exemplar realms
        result=get_subcategories_for_a_realm(realm,is_bio_realm)
        results_dict[realm]=result.replace(".","").split(", ")
    all_df=[]
    for subtype in results_dict[realm][0:num_subtypes_each_realm]:
        print(realm,":", subtype)
        if is_bio_realm:
            system_prompt=biological_system_prompt.format(realm,classes_per_type_count_biological,max_description_chars)
        else:
            system_prompt=object_system_prompt.format(realm,classes_per_type_count_object,max_description_chars)
        df=get_classes_and_descriptions_from_subcategory(realm,subtype,system_prompt,is_bio_realm)
        df.insert(0,"subtype",subtype)
        df["SD_prompt"]=""
        for idx in range(df.shape[0]):
            if is_bio_realm:
                df.at[idx,"SD_prompt"]="A photograph of a {} ({}): {}".format(df.iloc[idx]["species_common_name"],df.iloc[idx]["species_latin_name"],df.iloc[idx]["species_description"])
            else:
                df.at[idx,"SD_prompt"]="A photograph of {}: {}".format(df.iloc[idx]["item_name"],df.iloc[idx]["item_description"])
        all_df.append(df)
    all_df=pd.concat(all_df).reset_index(drop=True)
    del all_df["item_id"]
    #remove duplicates
    if is_bio_realm:
        all_df=all_df.drop_duplicates(subset=['species_common_name'], keep='first')
        all_df=all_df.drop_duplicates(subset=['species_latin_name'], keep='first')
    else:
        all_df=all_df.drop_duplicates(subset=['item_name'], keep='first')
    return all_df 


## Given the exemplars above, we won't do the initial set of prompting for Birds and Food
- simply use the exemplars

In [9]:
results_dict=dict()
results_dict["Birds"]=exemplar_bird_orders.split(', ')
results_dict["Food"]=exemplar_food_types.split(', ')

## Add to the two exemplars wwith four more *realms* as our examples:

In [10]:
realms_for_prompting=exemplar_realms+["Items found near roads", "Domestic furniture", "Plants", "Insects"]
biological_realms=["Birds","Plants","Insects"]

## For each given realm, generate classes and Stable Diffusion prompts for each class

In [11]:
for idx,realm in enumerate(realms_for_prompting):
    is_bio = True if realm in biological_realms else False
    classes_df=get_classes_from_realm(realm,is_bio)
    classes_df.to_csv(realm+".csv",index=False)
    display(classes_df)
    print("")
    

Birds : casuariiformes
Birds : tinamiformes


,subtype,species_latin_name,species_common_name,species_description,SD_prompt
0,casuariiformes,Casuarius casuarius,Southern Cassowary,"Large bird with a vivid blue face and neck, tw...",A photograph of a Southern Cassowary (Casuariu...
1,casuariiformes,Casuarius unappendiculatus,Northern Cassowary,"Large, black bird with a blue neck and one or ...",A photograph of a Northern Cassowary (Casuariu...
2,casuariiformes,Casuarius bennetti,Dwarf Cassowary,"Smaller than other cassowaries, with a black b...",A photograph of a Dwarf Cassowary (Casuarius b...
3,casuariiformes,Dromaius novaehollandiae,Emu,"Tall, flightless bird with long, shaggy, brown...",A photograph of a Emu (Dromaius novaehollandia...
4,casuariiformes,Apteryx australis,Southern Brown Kiwi,"Small, flightless bird with shaggy, brown feat...",A photograph of a Southern Brown Kiwi (Apteryx...
5,casuariiformes,Apteryx haastii,Great Spotted Kiwi,"Larger kiwi with grey-brown plumage, spotted w...",A photograph of a Great Spotted Kiwi (Apteryx ...
6,casuariiformes,Apteryx owenii,Little Spotted Kiwi,"The smallest kiwi, with fine, hair-like feathe...",A photograph of a Little Spotted Kiwi (Apteryx...
7,casuariiformes,Apteryx rowi,Okarito Kiwi,A rare kiwi with greyish-brown plumage and a l...,A photograph of a Okarito Kiwi (Apteryx rowi):...
8,tinamiformes,Tinamus major,Great Tinamou,Dark greyish-blue bird with a long neck and sm...,A photograph of a Great Tinamou (Tinamus major...
9,tinamiformes,Tinamus osgoodi,Black Tinamou,"A large, dark bird with a long neck and small ...",A photograph of a Black Tinamou (Tinamus osgoo...



Food : Appetizer
Food : Main course


,subtype,item_name,item_description,SD_prompt
0,Appetizer,Bruschetta,"Toasted bread topped with tomatoes, basil, gar...",A photograph of Bruschetta: Toasted bread topp...
1,Appetizer,Spring Rolls,"Thin pastry filled with vegetables or meat, ro...",A photograph of Spring Rolls: Thin pastry fill...
2,Appetizer,Caprese Skewers,"Skewers of cherry tomatoes, mozzarella balls, ...",A photograph of Caprese Skewers: Skewers of ch...
3,Appetizer,Deviled Eggs,"Hard-boiled eggs cut in half, filled with a cr...",A photograph of Deviled Eggs: Hard-boiled eggs...
4,Appetizer,Shrimp Cocktail,"Chilled, cooked shrimp served with a tangy tom...","A photograph of Shrimp Cocktail: Chilled, cook..."
5,Appetizer,Stuffed Mushrooms,Mushrooms filled with a mixture of breadcrumbs...,A photograph of Stuffed Mushrooms: Mushrooms f...
6,Appetizer,Nachos,"Tortilla chips layered with cheese, jalapenos,...",A photograph of Nachos: Tortilla chips layered...
7,Appetizer,Meatballs,"Small balls of ground meat, often seasoned and...",A photograph of Meatballs: Small balls of grou...
8,Appetizer,Mini Quiches,"Small, bite-sized quiches with a variety of fi...","A photograph of Mini Quiches: Small, bite-size..."
9,Appetizer,Crostini,Small slices of grilled or toasted bread toppe...,A photograph of Crostini: Small slices of gril...



Items found near roads : Street signs
Items found near roads : traffic lights


...

,subtype,item_name,item_description,SD_prompt
0,Sofa,Chesterfield Sofa,"Characterized by deep button tufting, rolled a...",A photograph of Chesterfield Sofa: Characteriz...
1,Sofa,Cabriole Sofa,"Features an exposed wooden frame, continuous c...",A photograph of Cabriole Sofa: Features an exp...
2,Sofa,Camelback Sofa,Identified by its arched back that peaks in th...,A photograph of Camelback Sofa: Identified by ...
3,Sofa,Lawson Sofa,"Designed for comfort, with a high back, low ar...",A photograph of Lawson Sofa: Designed for comf...
4,Sofa,Tuxedo Sofa,"Defined by its boxy shape, high arms level wit...",A photograph of Tuxedo Sofa: Defined by its bo...
5,Sofa,English Rolled Arm Sofa,"Known for low arms in relation to a high back,...",A photograph of English Rolled Arm Sofa: Known...
6,Sofa,Bridgewater Sofa,Casual and comfortable with softly rolled back...,A photograph of Bridgewater Sofa: Casual and c...
7,Sofa,Mid-Century Modern Sofa,"Features clean lines, geometric shapes, and fu...",A photograph of Mid-Century Modern Sofa: Featu...
8,Sofa,Sectional Sofa,"Modular, often L-shaped or U-shaped, and comes...","A photograph of Sectional Sofa: Modular, often..."
9,Sofa,Sleeper Sofa,A functional sofa that can be converted into a...,A photograph of Sleeper Sofa: A functional sof...



Plants : Acorales
Plants : Alismatales


,subtype,species_latin_name,species_common_name,species_description,SD_prompt
0,Acorales,Acorus calamus,Sweet Flag,"Long, sword-shaped green leaves with a promine...",A photograph of a Sweet Flag (Acorus calamus):...
1,Acorales,Acorus gramineus,Grass-leaved Sweet Flag,"Shorter, grass-like green leaves, often used i...",A photograph of a Grass-leaved Sweet Flag (Aco...
2,Alismatales,Echinodorus grandiflorus,Large-flowered Amazon Sword,"Large rosette plant with broad, green leaves a...",A photograph of a Large-flowered Amazon Sword ...
3,Alismatales,Sagittaria sagittifolia,Arrowhead Plant,Aquatic plant with arrow-shaped leaves and sma...,A photograph of a Arrowhead Plant (Sagittaria ...
4,Alismatales,Zostera marina,Eelgrass,"Submerged marine plant with long, ribbon-like ...",A photograph of a Eelgrass (Zostera marina): S...
5,Alismatales,Alisma plantago-aquatica,European Water Plantain,"Aquatic plant with broad, lance-shaped leaves ...",A photograph of a European Water Plantain (Ali...
6,Alismatales,Butomus umbellatus,Flowering Rush,"Aquatic plant with long, narrow leaves and umb...",A photograph of a Flowering Rush (Butomus umbe...
7,Alismatales,Vallisneria spiralis,Italian Vallisneria,"Submerged aquatic plant with long, spiraling l...",A photograph of a Italian Vallisneria (Vallisn...
8,Alismatales,Aponogeton distachyos,Water Hawthorn,"Aquatic plant with floating, oval leaves and f...",A photograph of a Water Hawthorn (Aponogeton d...
9,Alismatales,Lemna minor,Common Duckweed,"Tiny, free-floating aquatic plant with small, ...",A photograph of a Common Duckweed (Lemna minor...



Insects : Blattodea
Insects : Coleoptera


,subtype,species_latin_name,species_common_name,species_description,SD_prompt
0,Blattodea,Periplaneta americana,American Cockroach,"Large, reddish-brown with a yellowish figure 8...",A photograph of a American Cockroach (Periplan...
1,Blattodea,Blatta orientalis,Oriental Cockroach,"Shiny, dark brown to black and about 1 inch in...",A photograph of a Oriental Cockroach (Blatta o...
2,Blattodea,Blattella germanica,German Cockroach,"Small, light brown with two dark stripes runni...",A photograph of a German Cockroach (Blattella ...
3,Blattodea,Supella longipalpa,Brown-banded Cockroach,"Small, light brown with two lighter bands acro...",A photograph of a Brown-banded Cockroach (Supe...
4,Blattodea,Nauphoeta cinerea,Lobster Cockroach,"Medium-sized, mottled brown with a lobster-lik...",A photograph of a Lobster Cockroach (Nauphoeta...
5,Blattodea,Gromphadorhina portentosa,Madagascar Hissing Cockroach,"Large, brown, and oval-shaped, known for its h...",A photograph of a Madagascar Hissing Cockroach...
6,Blattodea,Eublaberus posticus,Orange Head Roach,"Medium-sized, dark brown with a distinctive or...",A photograph of a Orange Head Roach (Eublaberu...
7,Blattodea,Lucihormetica verrucosa,Warty Glowspot Cockroach,"Medium-sized, dark brown with luminescent spot...",A photograph of a Warty Glowspot Cockroach (Lu...
8,Blattodea,Blaberus discoidalis,Discoid Cockroach,"Large, dark brown with a slightly flattened bo...",A photograph of a Discoid Cockroach (Blaberus ...
9,Blattodea,Periplaneta australasiae,Australian Cockroach,"Medium-sized, brown with a yellow margin on th...",A photograph of a Australian Cockroach (Peripl...


## Display an example Stable Diffusion prompt for each example *realm*:

In [12]:
for idx,realm in enumerate(realms_for_prompting):
    classes_df=pd.read_csv(realm+".csv")
    print("An example prompt for the realm '{}':".format(realm))
    print("    - ",classes_df.at[classes_df.shape[0]-1,"SD_prompt"],"\n")

An example prompt for the realm 'Birds':
    -  A photograph of a Barred Tinamou (Crypturellus casiquiare): A medium-sized bird with a barred brown body, long neck and small head. 

An example prompt for the realm 'Food':
    -  A photograph of Fish and Chips: A British dish consisting of fried battered fish and hot potato chips, often served with mushy peas. 

An example prompt for the realm 'Items found near roads':
    -  A photograph of Bus Priority Signal: Specific signals that allow buses to move through intersections more quickly, often showing a white 'B' on a black background. 

An example prompt for the realm 'Domestic furniture':
    -  A photograph of Chaise Lounge Armchair: Long chair allowing for leg extension, often used for relaxation or reading in living spaces. 

An example prompt for the realm 'Plants':
    -  A photograph of a Paddle Weed (Halophila ovalis): Marine seagrass with small, oval leaves and tiny, white flowers. 

An example prompt for the realm 'Insects':

## The resulting prompts stored in csv file are now ready for passing to Stable Diffusion
- For our reports experiments, we omitted the limit imposed by "num_subtypes_each_realm=2" used here and instead used all subtypes returned by the initial prompt of the LLM